In [8]:
import json

# import the smartpynector module
from NBs import smartpynector as sp

In [9]:
# Defining constants
# Knowledge engine REST API URL
KNOWLEDGE_ENGINE_URL = "http://127.0.0.1:8280/rest"
SUBSCRIBER_ID = "http://127.0.0.1:8000/subscriber"

In [10]:
# Defining the smart connector
smart_connector = sp.SmartConnector(
    knowledgeBaseId=SUBSCRIBER_ID,
    knowledgeBaseName="Subscriber2",
    knowledgeBaseDescription="Subscriber2",
    reasonerEnabled=False,
)

In [11]:
sp.create_smart_connector(smart_connector, KNOWLEDGE_ENGINE_URL)

2023-05-12 16:19:28 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-12 16:19:30 DEBUG http://127.0.0.1:8280 "POST /rest/sc HTTP/1.1" 200 0
2023-05-12 16:19:30 INFO Request successful
2023-05-12 16:19:30 DEBUG 


<Response [200]>

In [12]:
PREFIXES={
        "rdf": "http://www.w3.org/1999/02/22-rdf-syntax-ns#",
        "saref": "https://w3id.org/saref#",
    }
GRAPH_PATTERN= '''?meas rdf:type saref:Measurement .
         ?meas saref:hasValue ?temp .
         ?meas saref:isMeasuredIn saref:TemperatureUnit .
         ?meas saref:hasTimestamp ?timestamp .
         ?meas saref:isMeasurementOf ?room_id .
         ?meas saref:relatesToProperty saref:Temperature .
         ?meas saref:measurementMadeBy ?device_id .'''

In [13]:
ask_ki_id = sp.register_knowledge_interaction(
    knowledge_interaction_type="AskKnowledgeInteraction",
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    prefixes=PREFIXES,
    graph_pattern=GRAPH_PATTERN,
    headers={
        "Content-Type": "application/json",
        "knowledge-Base-Id": SUBSCRIBER_ID,
    },
).json()["knowledgeInteractionId"]
ask_ki_id

2023-05-12 16:19:30 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-12 16:19:30 DEBUG http://127.0.0.1:8280 "POST /rest/sc/ki HTTP/1.1" 200 110
2023-05-12 16:19:30 INFO Request successful
2023-05-12 16:19:30 DEBUG {"knowledgeInteractionId":"http://127.0.0.1:8000/subscriber/interaction/ff576270-cd50-4fc8-acd8-7db66adbb5d9"}


'http://127.0.0.1:8000/subscriber/interaction/ff576270-cd50-4fc8-acd8-7db66adbb5d9'

In [14]:
binding_set = sp.perform_ask_knowledge_interaction(
    knowledge_engine_url=KNOWLEDGE_ENGINE_URL,
    knowledge_base_id=SUBSCRIBER_ID,
    knowledge_interaction_id=ask_ki_id,
    ask_data=[]
).json()["bindingSet"]

2023-05-12 16:19:30 DEBUG Starting new HTTP connection (1): 127.0.0.1:8280
2023-05-12 16:19:30 DEBUG http://127.0.0.1:8280 "POST /rest/sc/ask HTTP/1.1" 200 895
2023-05-12 16:19:30 INFO Request successful
2023-05-12 16:19:30 DEBUG {"bindingSet":[{"room_id":"\"http://127.0.0.1:8000/thermostat/rooms/1\"","temp":"20","device_id":"\"http://127.0.0.1:8000/thermostat/devices/1\"","meas":"<http://127.0.0.1:8000/thermostat/measurements/c84baa72-3d80-42dd-8caf-bc17aa803043>","timestamp":"\"2023-05-12T16:19:27+02:00\""}],"exchangeInfo":[{"initiator":"knowledgeBase","knowledgeBaseId":"http://127.0.0.1:8000/thermostat","knowledgeInteractionId":"http://127.0.0.1:8000/thermostat/interaction/be4fdc09-a3a1-46d5-b6ad-e802510e167d","exchangeStart":"2023-05-12T14:19:30.749+00:00","exchangeEnd":"2023-05-12T14:19:30.937+00:00","status":"SUCCEEDED","bindingSet":[{"room_id":"\"http://127.0.0.1:8000/thermostat/rooms/1\"","temp":"20","device_id":"\"http://127.0.0.1:8000/thermostat/devices/1\"","meas":"<http://1

In [15]:
READ_URL = "https://graphdb.odissei.nl/repositories/MateuszTest"
WRITE_URL = "https://graphdb.odissei.nl/repositories/MateuszTest/statements"

sp.store_data_in_graphdb(
    graph_pattern=GRAPH_PATTERN,
    binding_set=binding_set,
    prefixes=PREFIXES,
    read_url=READ_URL,
    write_url=WRITE_URL,
)

2023-05-12 16:19:32 DEBUG Ignoring 500 server error
